In [ ]:
import os
from getpass import getpass
user = input('GitHub user ')
password = getpass('GitHub password ')
os.environ['GITHUB_AUTH'] = user + ':' + password
!git clone https://$GITHUB_AUTH@github.com/romanenkova95/semi_supervised_CPD.git
!cd semi_supervised_CPD && git checkout alpha

Cloning into 'semi_supervised_CPD'...
remote: Enumerating objects: 161643, done.
remote: Counting objects: 100% (295/295), done.
remote: Compressing objects: 100% (224/224), done.
remote: Total 161643 (delta 67), reused 281 (delta 62), pack-reused 161348
Receiving objects: 100% (161643/161643), 140.49 MiB | 26.07 MiB/s, done.
Resolving deltas: 100% (72/72), done.
Checking out files: 100% (158866/158866), done.
Branch 'alpha' set up to track remote branch 'alpha' from 'origin'.
Switched to a new branch 'alpha'


In [ ]:
%cd semi_supervised_CPD

/content/semi_supervised_CPD


In [ ]:
!pip install pytorch_lightning
!pip install pims
!pip install ruptures
!pip install av
!pip install pytorchvideo

# Supervised TS-CP baseline

In [ ]:
from BASELINES.tscp import *
from BASELINES import tscp
from BASELINES import models as tscp_encoders
from CPD import loss, train
from CPD import models as models_original
from CPD import datasets as datasets_original
from CPD import new_metrics_2 as metrics
from Semi_Supervised_TSCP import models
from Semi_Supervised_TSCP import datasets

import datetime
import torch
import numpy as np
from torch import nn

import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/_functional_video.py:7: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms.functional' module instead.
  "The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in 0.14. "
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/_transforms_video.py:26: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms' module instead.
  "The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in 0.14. "


# MNIST Experiments

In [ ]:
experiments_name = 'mnist'

## Datasets

In [ ]:
supervised_num = 6
unsupervised_num = 6

In [ ]:
supervised_train_dataset, supervised_test_dataset = datasets.CPDDatasetsNew(num=supervised_num, 
                                                                            num_change=supervised_num//2,
                                                                            path_to_data='data/mnist',
                                                                            experiments_name=experiments_name).get_dataset_()
unsupervised_train_dataset, unsupervised_test_dataset = datasets.CPDDatasetsNew(num=unsupervised_num,
                                                                                num_change=unsupervised_num//2,
                                                                                path_to_data='data/mnist',
                                                                                experiments_name=experiments_name).get_dataset_()

## Models

In [ ]:
SEQ_LEN = 16
batch_size = 4
pos_numbers = 2
neg_numbers = 2

alpha = 0.5

model = tscp_encoders.MNISTTSCPEncoder(seq_len=SEQ_LEN, c_in=784, 
                                      nb_filters=256, 
                                      code_size=8, 
                                      n_steps=128)

tscp_model = models.TSCPModelAlpha(model=model, supervised_train_dataset=supervised_train_dataset,
                                   supervised_test_dataset=supervised_test_dataset, 
                                   unsupervised_train_dataset=unsupervised_train_dataset,
                                   unsupervised_test_dataset=unsupervised_test_dataset,
                                   batch_size=batch_size, 
                                   window_1=SEQ_LEN, window_2=SEQ_LEN,
                                   pos_numbers=pos_numbers, neg_numbers=neg_numbers, 
                                   alpha=alpha)

## Run experiments

### CPD model with original datasets -- for validation

In [ ]:
cpd_model = models_original.CPD_model(experiments_name=experiments_name, loss_type='CPD', lr=10e-4)

### Training

In [ ]:
seed = 1
tscp_model = train.train_model(tscp_model, experiments_name, max_epochs=100,
                               patience=5, gpus=0, seed=seed, 
                               min_delta=1e-4, 
                               monitor='train_loss_epoch',
                               check_val_every_n_epoch=2)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | MNISTTSCPEncoder | 4.9 M 
-------------------------------------------
4.9 M     Trainable params
0         Non-trainable params
4.9 M     Total params
19.422    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

<class 'torch.Tensor'>
<class 'torch.Tensor'>


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Metric train_loss_epoch improved. New best score: 0.289


<class 'torch.Tensor'>
<class 'torch.Tensor'>


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Metric train_loss_epoch improved by 0.101 >= min_delta = 0.0001. New best score: 0.188


<class 'torch.Tensor'>


In [ ]:
%reload_ext tensorboard
%tensorboard --logdir 'logs'

### Look at the predictions

In [ ]:
#device = 'cuda:0'
device = 'cpu'
tscp_model.to(device)

for inputs, labels in cpd_model.val_dataloader():
    preds = tscp.get_tscp_output_2(tscp_model, inputs.to(device), window=tscp_model.window_1).detach().cpu()
    inputs.detach().cpu()
    labels.detach().cpu()
    for i in range(5):
        plt.figure()
        plt.plot(preds[i], label='pred')
        plt.plot(labels[i], label='true labels')
        plt.legend()

## Calculate metrics

In [ ]:
threshold_number = 45
threshold_list = np.linspace(-5, 5, threshold_number)
threshold_list = 1 / (1 + np.exp(-threshold_list))
threshold_list = [-0.001] + list(threshold_list) + [1.001]

# cut threshold list
threshold_list = list(threshold_list)[18:27]

In [ ]:
# evaluating on original val dataset
r, delay_list, fp_delay_list = metrics.evaluation_pipeline(tscp_model,
                                                           cpd_model.val_dataloader(),  
                                                           threshold_list, 
                                                           device=device, 
                                                           model_type='tscp',
                                                           verbose=True)

## FINETUNING

In [ ]:
supervised_num = 100
unsupervised_num = 500

In [ ]:
supervised_train_dataset, supervised_test_dataset = datasets.CPDDatasetsNew(num=supervised_num, 
                                                                            num_change=supervised_num//2,
                                                                            experiments_name=experiments_name).get_dataset_()
unsupervised_train_dataset, unsupervised_test_dataset = datasets.CPDDatasetsNew(num=unsupervised_num,
                                                                                num_change=unsupervised_num//2,
                                                                                experiments_name=experiments_name).get_dataset_()

In [ ]:
SEQ_LEN = 16
batch_size = 4
pos_numbers = 2
neg_numbers = 2

model = models.SyntheticTSCPEncoder(seq_len=SEQ_LEN, c_in=1, 
                                    nb_filters=4, 
                                    code_size=4, 
                                    n_steps=4)

tscp_model_unsupervised = models.TSCPModelAlpha(model=model, supervised_train_dataset=supervised_train_dataset,
                                                supervised_test_dataset=supervised_test_dataset, 
                                                unsupervised_train_dataset=unsupervised_train_dataset,
                                                unsupervised_test_dataset=unsupervised_test_dataset,
                                                batch_size=batch_size, 
                                                window_1=SEQ_LEN, window_2=SEQ_LEN,
                                                pos_numbers=pos_numbers, neg_numbers=neg_numbers, 
                                                alpha=1)
tscp_model_supervised = models.TSCPModelAlpha(model=tscp_model_unsupervised.model, 
                                              supervised_train_dataset=supervised_train_dataset,
                                              supervised_test_dataset=supervised_test_dataset, 
                                              unsupervised_train_dataset=unsupervised_train_dataset,
                                              unsupervised_test_dataset=unsupervised_test_dataset,
                                              batch_size=batch_size, 
                                              window_1=SEQ_LEN, window_2=SEQ_LEN,
                                              pos_numbers=pos_numbers, neg_numbers=neg_numbers, 
                                              alpha=0)

In [ ]:
seed = 0
tscp_model_unsupervised = train.train_model(tscp_model_unsupervised, 
                                            experiments_name, max_epochs=100,
                                            patience=5, gpus=0, seed=seed, 
                                            min_delta=1e-4, 
                                            monitor='train_loss_epoch',
                                            check_val_every_n_epoch=2)

tscp_model_supervised = train.train_model(tscp_model_supervised, 
                                          experiments_name, max_epochs=10,
                                          patience=5, gpus=0, seed=seed, 
                                          min_delta=1e-4, 
                                          monitor='train_loss_epoch',
                                          check_val_every_n_epoch=2)

In [ ]:
threshold_number = 45
threshold_list = np.linspace(-5, 5, threshold_number)
threshold_list = 1 / (1 + np.exp(-threshold_list))
threshold_list = [-0.001] + list(threshold_list) + [1.001]

# cut threshold list
threshold_list = list(threshold_list)[18:27]

In [ ]:
# evaluating on original val dataset
device='cpu'
r, delay_list, fp_delay_list = metrics.evaluation_pipeline(tscp_model_supervised,
                                                           cpd_model.val_dataloader(),  
                                                           threshold_list, 
                                                           device=device, 
                                                           model_type='tscp',
                                                           verbose=True)

In [ ]:
seed=1
SEQ_LEN = 16
batch_size = 4
pos_numbers = 2
neg_numbers = 2

model = tscp_encoders.MNISTTSCPEncoder(seq_len=SEQ_LEN, c_in=784, 
                                           nb_filters=256, 
                                           code_size=8, 
                                           n_steps=128)

tscp_model_unsupervised = models.TSCPModelAlpha(model=model, supervised_train_dataset=supervised_train_dataset,
                                                    supervised_test_dataset=supervised_test_dataset, 
                                                    unsupervised_train_dataset=unsupervised_train_dataset,
                                                    unsupervised_test_dataset=unsupervised_test_dataset,
                                                    batch_size=batch_size, 
                                                    window_1=SEQ_LEN, window_2=SEQ_LEN,
                                                    pos_numbers=pos_numbers, neg_numbers=neg_numbers, 
                                                    alpha=1)

tscp_model_unsupervised = train.train_model(tscp_model_unsupervised, 
                                            experiments_name, max_epochs=40,
                                            patience=5, gpus=0, seed=seed, 
                                            min_delta=1e-4, 
                                            monitor='train_loss_epoch',
                                            check_val_every_n_epoch=2)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | MNISTTSCPEncoder | 4.9 M 
-------------------------------------------
4.9 M     Trainable params
0         Non-trainable params
4.9 M     Total params
19.422    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Metric train_loss_epoch improved. New best score: 0.168


...

INFO:pytorch_lightning.callbacks.early_stopping:Metric train_loss_epoch improved by 0.002 >= min_delta = 0.0001. New best score: 0.043


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.callbacks.early_stopping:Metric train_loss_epoch improved by 0.001 >= min_delta = 0.0001. New best score: 0.042


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=40` reached.


In [ ]:
device='cpu'
r, delay_list, fp_delay_list = metrics.evaluation_pipeline(tscp_model_unsupervised,
                                                           cpd_model.val_dataloader(),  
                                                           threshold_list, 
                                                           device=device, 
                                                           model_type='tscp',
                                                           verbose=False)
print(r[6])
r

100%|██████████| 9/9 [11:28<00:00, 76.47s/it]

0.8066630523557652


(0.33585562413702275,
 45.69333267211914,
 6.813333511352539,
 326.7924264129633,
 (102, 81, 22, 95),
 0.6484641638225256,
 0.8066630523557652,
 0.44342513010268925,
 0.8110388004383657)